In [48]:
import baostock as bs
import pandas as pd
import time
import numpy as np
from datetime import datetime, timedelta

In [49]:
data = pd.read_csv("all_hs300_stocks.csv")

In [50]:
data['date'] = pd.to_datetime(data['date'])
data.set_index(['date'], inplace=True)
data

,code,open,close,high,low,volume
date,,,,,,
2008-01-02,sh.600000,53.00,53.55,55.05,51.81,13158390.0
2008-01-02,sh.600001,8.12,8.38,8.60,7.95,62828328.0
2008-01-02,sh.600003,6.21,6.21,6.21,6.21,0.0
2008-01-02,sh.600004,20.95,22.15,22.20,20.95,10329598.0
2008-01-02,sh.600005,19.79,19.30,19.89,19.00,26984436.0
...,...,...,...,...,...,...
2022-12-30,sz.300896,568.09,566.35,570.60,557.00,1286736.0
2022-12-30,sz.300919,68.50,65.61,69.12,65.61,7426278.0
2022-12-30,sz.300957,151.00,149.24,151.70,146.89,3534444.0


In [51]:
# 获取数据集中最早和最晚的日期
start_date = '2008-01-01'
end_date = '2022-12-31'

# 生成完整的日期范围，假设工作日是频率
complete_date_range = pd.date_range(start=start_date, end=end_date, freq='D')  # freq='B' 表示只包含工作日

# 获取实际存在的日期
actual_dates = data.index.unique()

# 找出缺失的日期
missing_dates = complete_date_range.difference(actual_dates)

# 打印缺失的日期
print("Missing dates:")
print(missing_dates)

Missing dates:
DatetimeIndex(['2008-01-01', '2008-01-05', '2008-01-06', '2008-01-12',
               '2008-01-13', '2008-01-19', '2008-01-20', '2008-01-26',
               '2008-01-27', '2008-02-02',
               ...
               '2022-11-27', '2022-12-03', '2022-12-04', '2022-12-10',
               '2022-12-11', '2022-12-17', '2022-12-18', '2022-12-24',
               '2022-12-25', '2022-12-31'],
              dtype='datetime64[ns]', length=1860, freq=None)


In [58]:
not_connect_date = []
bs.login()
for date in missing_dates:
    date = date.strftime("%Y-%m-%d")
    print(date)

    rs = bs.query_hs300_stocks(date=date)
    if rs.error_code == '0':
        hs300_stocks = []
        while rs.next():
            stock_data = rs.get_row_data()
            stock_code = stock_data[1]  # 假设股票代码在第二列
            # 查询股票的每日信息
            k_rs = bs.query_history_k_data_plus(stock_code,
                                                    "date,code,open,close,high,low,volume",
                                                    start_date=date, end_date=date,
                                                    frequency="d", adjustflag="3")
            if k_rs.error_code == '0':
                k_data = k_rs.get_row_data()
                if k_data == []:
                    break
                else:
                    # 添加open, close, high, low, volume信息
                    not_connect_date.append(date)
                    break
bs.logout()                    


login success!
2008-01-01
2008-01-05
2008-01-06
2008-01-12
2008-01-13
2008-01-19
2008-01-20
2008-01-26
2008-01-27
2008-02-02
2008-02-03
2008-02-06
2008-02-07
2008-02-08
2008-02-09
2008-02-10
2008-02-11
2008-02-12
2008-02-16
2008-02-17
2008-02-23
2008-02-24
2008-03-01
2008-03-02
2008-03-08
2008-03-09
2008-03-15
2008-03-16
2008-03-22
2008-03-23
2008-03-29
2008-03-30
2008-04-04
2008-04-05
2008-04-06
2008-04-12
2008-04-13
2008-04-19
2008-04-20
2008-04-26
2008-04-27
2008-05-01
2008-05-02
2008-05-03
2008-05-04
2008-05-10
2008-05-11
2008-05-17
2008-05-18
2008-05-24
2008-05-25
2008-05-31
2008-06-01
2008-06-07
2008-06-08
2008-06-09
2008-06-14
2008-06-15
2008-06-19
2008-06-21
2008-06-22
2008-06-28
2008-06-29
2008-07-05
2008-07-06
2008-07-12
2008-07-13
2008-07-19
2008-07-20
2008-07-26
2008-07-27
2008-08-02
2008-08-03
2008-08-09
2008-08-10
2008-08-16
2008-08-17
2008-08-23
2008-08-24
2008-08-30
2008-08-31
2008-09-06
2008-09-07
2008-09-13
2008-09-14
2008-09-15
2008-09-20
2008-09-21
2008-09-27
2008-0

In [59]:
not_connect_date

['2008-06-19',
 '2009-03-03',
 '2009-08-21',
 '2010-02-08',
 '2010-04-21',
 '2010-05-05',
 '2011-04-12',
 '2011-06-23',
 '2011-07-07',
 '2012-06-13',
 '2012-09-07',
 '2013-02-25',
 '2013-05-22',
 '2014-01-24',
 '2014-04-29',
 '2014-07-24',
 '2015-07-01',
 '2015-09-25',
 '2016-03-14',
 '2016-09-01',
 '2017-05-16',
 '2017-11-03',
 '2018-07-18',
 '2018-10-12',
 '2019-04-01',
 '2019-09-19',
 '2020-06-02',
 '2020-11-20',
 '2021-08-04',
 '2022-04-18']

In [61]:
all_hs300_stocks = pd.DataFrame()

    # 登陆系统
    

#current_date = start_date
for current_date in not_connect_date:
    lg = bs.login()
    current_date = datetime.strptime(current_date, '%Y-%m-%d')
    if lg.error_code != '0':
        print(f"Login failed with error: {lg.error_msg}")
    print(f"Querying date {current_date.strftime('%Y-%m-%d')}")
    date_str = current_date.strftime("%Y-%m-%d")
    rs = bs.query_hs300_stocks(date=date_str)
    if rs.error_code == '0':
        hs300_stocks = []
        while rs.next():
            stock_data = rs.get_row_data()
            stock_code = stock_data[1]  # 假设股票代码在第二列
            # 查询股票的每日信息
            k_rs = bs.query_history_k_data_plus(stock_code,
                                                    "date,code,open,close,high,low,volume",
                                                    start_date=date_str, end_date=date_str,
                                                    frequency="d", adjustflag="3")
            if k_rs.error_code == '0':
                k_data = k_rs.get_row_data()
                if k_data == []:
                    break
                else:
                        # 添加open, close, high, low, volume信息
                    hs300_stocks.append(k_data)
        if hs300_stocks:
            result = pd.DataFrame(hs300_stocks, columns=['date',
                                                             'code',
                                                             'open','close','high','low','volume'])
            all_hs300_stocks = pd.concat([all_hs300_stocks, result])
    else:
        print(f"Error querying date {date_str}: {rs.error_msg}")

login success!
Querying date 2008-06-19
login success!
Querying date 2009-03-03
login success!
Querying date 2009-08-21
login success!
Querying date 2010-02-08
login success!
Querying date 2010-04-21
login success!
Querying date 2010-05-05
login success!
Querying date 2011-04-12
login success!
Querying date 2011-06-23
login success!
Querying date 2011-07-07
login success!
Querying date 2012-06-13
login success!
Querying date 2012-09-07
login success!
Querying date 2013-02-25
login success!
Querying date 2013-05-22
login success!
Querying date 2014-01-24
login success!
Querying date 2014-04-29
login success!
Querying date 2014-07-24
login success!
Querying date 2015-07-01
login success!
Querying date 2015-09-25
login success!
Querying date 2016-03-14
login success!
Querying date 2016-09-01
login success!
Querying date 2017-05-16
login success!
Querying date 2017-11-03
login success!
Querying date 2018-07-18
login success!
Querying date 2018-10-12
login success!
Querying date 2019-04-01


In [63]:
all_hs300_stocks.to_csv("/home/cseadmin/mz/StockMtsPlatform/get_csi_data/all_hs300_stocks_not_connect.csv", encoding="gbk", index=False)

In [42]:
idx = pd.IndexSlice

# 获取从 '2020-01-02' 到 '2020-01-05' 的所有股票数据
data.loc[idx['2020-01-02':'2020-01-05', :]]

open  close   high    low       volume
date       code                                              
2020-01-02 sh.600000  12.47  12.47  12.64  12.45   51629079.0
           sh.600004  17.56  17.52  17.62  17.40   17792404.0
           sh.600009  78.78  77.36  78.91  77.10   15826563.0
           sh.600010   1.33   1.33   1.34   1.32  204217823.0
           sh.600011   5.61   5.64   5.69   5.60   47080324.0
...                     ...    ...    ...    ...          ...
2020-01-03 sz.300347  62.70  61.79  63.00  61.31    5425179.0
           sz.300408  23.61  23.77  24.80  23.47   25750191.0
           sz.300413  38.80  39.35  40.33  37.91   25143321.0
           sz.300433  14.80  15.16  15.25  14.66   78707262.0
           sz.300498  34.39  34.60  34.76  33.95   28143696.0

[600 rows x 5 columns]